In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_1.csv')

In [31]:
main_data_df = main_data_df[main_data_df['대장구분명'] == '일반']
main_data_df.shape

(4553162, 17)

In [32]:
nan_df = main_data_df[main_data_df['도로명주소'].isna()]
nan_df[['지번주소', '도로명주소']].isna().sum()

지번주소         0
도로명주소    46330
dtype: int64

In [33]:
addr_to_download = nan_df['지번주소'].unique().tolist()
len(addr_to_download)

1644

In [36]:
# download coordinates from an open api

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    addr_df = pd.DataFrame({'지번주소':[], '도로명주소': []})
    for i, address in tqdm(enumerate(addr_list)):
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                       + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        if roadname_info == []:
            splitted = address_no_zero.split('-')
            space_splitted = address_no_zero.split(' ')
            
            if len(splitted) == 1:
                bonbun = int(space_splitted[3])
                bonbun_positive = bonbun
                bonbun_negative = bonbun
                
                while roadname_info == []:
                
                    bonbun_positive += 1
                    bonbun_negative -= 1
                    
                    addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                               + addr + '&resultType=json' + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                   + addr + '&resultType=json' + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                
                
            else:
                bubun = int(splitted[1])
                bubun_positive = bubun
                bubun_negative = bubun
                
                count = 0
                while roadname_info == []:
                    
                    bubun_positive += 1
                    bubun_negative -= 1
                    
                    addr = splitted[0] + '-' + str(bubun_positive)
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                               + addr + '&resultType=json' + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso']
                    
                    if roadname_info == []:
                        
                        addr = splitted[0] + '-' + str(bubun_negative)
                        
                        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                   + addr + '&resultType=json' + '&hstryYn=Y')
                        roadname_info = roadname_result.json()['results']['juso']
                    
                    count += 1
                    
                    if count > 10:
                        
                        addr = splitted[0]
                        
                        space_splitted = addr.split(' ')
                        
                        bonbun = int(space_splitted[3])
                        bonbun_positive = bonbun
                        bonbun_negative = bonbun
                        
                        while roadname_info == []:

                            bonbun_positive += 1
                            bonbun_negative -= 1

                            addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_positive)
                            
                            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                       + addr + '&resultType=json' + '&hstryYn=Y')
                            roadname_info = roadname_result.json()['results']['juso']

                            if roadname_info == []:

                                addr = space_splitted[0] + ' ' + space_splitted[1] + ' ' + space_splitted[2] + ' ' + str(bonbun_negative)
                                
                                roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                                           + addr + '&resultType=json' + '&hstryYn=Y')
                                roadname_info = roadname_result.json()['results']['juso']
                                
                    
        
        
        roadname_addr = roadname_info[0]['roadAddrPart1']
        
        addr_df.loc[i,'지번주소'] = address
        addr_df.loc[i,'도로명주소'] = roadname_addr
    
    
    return addr_df

In [37]:
roadname_addr_df = fetch_coor_data(addr_to_download)

1644it [20:48,  1.32it/s]


In [38]:
roadname_addr_df.shape

(1644, 2)

In [39]:
roadname_addr_df.head()

,지번주소,도로명주소
0,서울특별시 강남구 논현동 276-0,서울특별시 강남구 언주로 604
1,서울특별시 강남구 대치동 633-0,서울특별시 강남구 삼성로51길 42
2,서울특별시 강남구 대치동 610-0,서울특별시 강남구 삼성로51길 35
3,서울특별시 강남구 도곡동 538-0,서울특별시 강남구 도곡로 320
4,서울특별시 강남구 신사동 506-7,서울특별시 강남구 압구정로2길 18-3


In [42]:
for i in tqdm(range(roadname_addr_df.shape[0])):
    addr_df = main_data_df[main_data_df['지번주소'] == roadname_addr_df.loc[i,'지번주소']]
    addr_df['도로명주소'] = roadname_addr_df.loc[i,'도로명주소']
    main_data_df.loc[addr_df.index,'도로명주소'] = addr_df['도로명주소']

100%|██████████████████████████████████████████████████████████████████████████████| 1644/1644 [05:44<00:00,  4.77it/s]


In [43]:
main_data_df['도로명주소'].isna().sum()

0

In [44]:
main_data_df.to_csv('./prepped_data/main_data_baseline_checkpoint_1.csv')